In [70]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

In [87]:
#Turns the CSV file into a pandas dataframe
df = pd.read_csv('CdSe - BetterthanRaw.csv')
df

,Temperature,Growth,Metal_source,Metal_amount (g),Metal_mmol (mmol),Metal_concentration (mmol/g),Chalcogen_source,Chalcogen_amount (g),Chalcogen_mmol (mmol),Chalcogen_concentration (mmol/g),...,Time_min (min),Purification antisolvent,Diameter_nm,FWHM (nm),Absorbance max (nm),PL max (nm),PLQY (%),Phase,Diameter from,Citation
0,225,225,cadmium oxide,0.0100,0.10,0.010,Se powder,0.0300,0.38,0.0400,...,0.50,NaN,2.5,NaN,474.0,617.0,None,NaN,TEM,Colloids and Surfaces A: Physicochemical and E...
1,255,255,cadmium oxide,0.0500,0.39,0.160,Se powder,0.0100,0.13,0.0500,...,0.17,methanol & chloroform,2.6,NaN,526.0,556.0,None,NaN,TEM,Colloids and Surfaces A: Physicochemical and E...
2,255,255,cadmium oxide,0.0500,0.39,0.160,Se powder,0.0100,0.13,0.0500,...,0.34,methanol & chloroform,3.2,NaN,559.0,580.0,None,NaN,TEM,Colloids and Surfaces A: Physicochemical and E...
3,255,255,cadmium oxide,0.0500,0.39,0.160,Se powder,0.0100,0.13,0.0500,...,0.68,methanol & chloroform,3.6,NaN,574.0,598.0,None,NaN,TEM,Colloids and Surfaces A: Physicochemical and E...
4,255,255,cadmium oxide,0.0500,0.39,0.160,Se powder,0.0100,0.13,0.0500,...,1.00,methanol & chloroform,3.9,NaN,582.0,601.0,None,NaN,TEM,Colloids and Surfaces A: Physicochemical and E...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,240,220,cadmium stearate,0.1846,0.27,0.082,Se powder,0.0213,2.70,0.8196,...,15.00,toluene,2.0,none,375.0,3.3,none,wurtzite,TEM,J. Am. Chem. Soc. 2005 127 12 4372 - 4375
108,240,220,cadmium stearate,0.1846,0.27,0.082,Se powder,0.0213,2.70,0.8196,...,30.00,toluene,2.0,none,375.0,3.5,none,wurtzite,TEM,J. Am. Chem. Soc. 2005 127 12 4372 - 4375
109,240,220,cadmium stearate,0.1846,0.27,0.082,Se powder,0.0213,2.70,0.8196,...,45.00,toluene,2.0,none,375.0,3.5,none,wurtzite,TEM,J. Am. Chem. Soc. 2005 127 12 4372 - 4375
110,240,220,cadmium stearate,0.1846,0.27,0.082,Se powder,0.0213,2.70,0.8196,...,60.00,toluene,2.0,none,375.0,3.4,none,wurtzite,TEM,J. Am. Chem. Soc. 2005 127 12 4372 - 4375


In [88]:
#Identifies the shape of the original dataframe
df.shape

(112, 36)

In [89]:
#Checks if there are any columns with no values
df.isna().sum()

Temperature                          0
Growth                               0
Metal_source                         0
Metal_amount (g)                     0
Metal_mmol (mmol)                    0
Metal_concentration (mmol/g)         0
Chalcogen_source                     0
Chalcogen_amount (g)                 0
Chalcogen_mmol (mmol)                0
Chalcogen_concentration (mmol/g)     0
Metal/Se_ratio                       0
Carboxylic_Acid                      0
CA_amount (g)                        0
CA_mmol (mmol)                       0
Cd/CA_ratio                          0
Amines                               0
Amines_amount (g)                    0
Phosphines                           0
Phosphines_amount (g)                0
Phosphines_mmol (mmol)               0
Chalcogen/Ph_ratio                   0
Solvent I                            0
S_I_amount (g)                       0
Solvent II                           0
S_II_amount (g)                      0
Total_amount (g)         

In [135]:
#Separate out initial DataFrame into the input features and output features
df_input = df.drop(columns =['Chalcogen_source','FWHM (nm)', 'Phase', 'Purification antisolvent', 'Diameter_nm', 'Absorbance max (nm)', 'PL max (nm)', 'PLQY (%)', 'Diameter from', 'Citation'], inplace = False, axis = 1) #Dropped out FWHM, Phase and PLQY columns as well because they have NaN values
df_output = df[['Diameter_nm', 'Absorbance max (nm)', 'PL max (nm)']]

In [136]:
#Checks the column names, and ensures that they do not have any leading or trailing spaces
df_input.columns = df_input.columns.str.strip()
df_output.columns = df_output.columns.str.strip()

In [137]:
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 26 columns):
Temperature                         112 non-null int64
Growth                              112 non-null int64
Metal_source                        112 non-null object
Metal_amount (g)                    112 non-null float64
Metal_mmol (mmol)                   112 non-null float64
Metal_concentration (mmol/g)        112 non-null float64
Chalcogen_amount (g)                112 non-null float64
Chalcogen_mmol (mmol)               112 non-null float64
Chalcogen_concentration (mmol/g)    112 non-null float64
Metal/Se_ratio                      112 non-null float64
Carboxylic_Acid                     112 non-null object
CA_amount (g)                       112 non-null float64
CA_mmol (mmol)                      112 non-null float64
Cd/CA_ratio                         112 non-null float64
Amines                              112 non-null object
Amines_amount (g)                   112 non-nul

In [138]:
#Converts the values in the "Temperature" and "Growth" Columns into float types
df_input['Temperature'] = df_input['Temperature'].astype(float)
df_input['Growth'] = df_input['Growth'].astype(float)

df_input.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 26 columns):
Temperature                         112 non-null float64
Growth                              112 non-null float64
Metal_source                        112 non-null object
Metal_amount (g)                    112 non-null float64
Metal_mmol (mmol)                   112 non-null float64
Metal_concentration (mmol/g)        112 non-null float64
Chalcogen_amount (g)                112 non-null float64
Chalcogen_mmol (mmol)               112 non-null float64
Chalcogen_concentration (mmol/g)    112 non-null float64
Metal/Se_ratio                      112 non-null float64
Carboxylic_Acid                     112 non-null object
CA_amount (g)                       112 non-null float64
CA_mmol (mmol)                      112 non-null float64
Cd/CA_ratio                         112 non-null float64
Amines                              112 non-null object
Amines_amount (g)                   112 non

In [139]:
#Initializes 2 lists to contain all of the numerical and categorical input columns
input_num_cols = [col for col in df_input.columns if df[col].dtypes !='O']
input_cat_cols = [col for col in df_input.columns if df[col].dtypes =='O']

In [256]:
#Initializes the ColumnTransformer object, and specifies what it will do with a passed in dataframe
ct = ColumnTransformer([
    ('step1', RobustScaler(), input_num_cols),
    ('step2', OneHotEncoder(sparse=False, handle_unknown='ignore'), input_cat_cols)], 
    remainder = 'passthrough')

In [257]:
#Uses the ColumnTransformer object to modify the input columns
df_input_scaled_encoded = pd.DataFrame(ct.fit_transform(df_input))
df_input_scaled_encoded

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,-0.539683,-0.08,-1.0000,-1.243697,-0.666667,0.000000,-0.111111,0.526316,-0.362745,-0.045437,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,-0.158730,0.40,-0.5000,-0.268908,4.333333,-0.365297,-0.419753,0.789474,1.024510,0.757289,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.158730,0.40,-0.5000,-0.268908,4.333333,-0.365297,-0.419753,0.789474,1.024510,0.757289,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.158730,0.40,-0.5000,-0.268908,4.333333,-0.365297,-0.419753,0.789474,1.024510,0.757289,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.158730,0.40,-0.5000,-0.268908,4.333333,-0.365297,-0.419753,0.789474,1.024510,0.757289,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,-0.349206,-0.16,1.1825,-0.672269,1.733333,-0.158904,2.753086,21.042105,-0.446078,-0.454373,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
108,-0.349206,-0.16,1.1825,-0.672269,1.733333,-0.158904,2.753086,21.042105,-0.446078,-0.454373,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
109,-0.349206,-0.16,1.1825,-0.672269,1.733333,-0.158904,2.753086,21.042105,-0.446078,-0.454373,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
110,-0.349206,-0.16,1.1825,-0.672269,1.733333,-0.158904,2.753086,21.042105,-0.446078,-0.454373,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


#### Now that we have transformed all of the relevant columns, how can we take back all of the relevant column names?

In [258]:
ct.transformers_

[('step1',
  RobustScaler(),
  ['Temperature',
   'Growth',
   'Metal_amount (g)',
   'Metal_mmol (mmol)',
   'Metal_concentration (mmol/g)',
   'Chalcogen_amount (g)',
   'Chalcogen_mmol (mmol)',
   'Chalcogen_concentration (mmol/g)',
   'Metal/Se_ratio',
   'CA_amount (g)',
   'CA_mmol (mmol)',
   'Cd/CA_ratio',
   'Amines_amount (g)',
   'Phosphines_amount (g)',
   'Phosphines_mmol (mmol)',
   'Chalcogen/Ph_ratio',
   'S_I_amount (g)',
   'S_II_amount (g)',
   'Total_amount (g)',
   'Time_min (min)']),
 ('step2',
  OneHotEncoder(handle_unknown='ignore', sparse=False),
  ['Metal_source',
   'Carboxylic_Acid',
   'Amines',
   'Phosphines',
   'Solvent I',
   'Solvent II'])]

In [259]:
#Look into ct object to see the column titles
array_of_cat_titles = ct.transformers_[1][1].get_feature_names()
len(array_of_cat_titles) #There are a total of 25 encoded categorical columns

24

In [260]:
#Number of numerical columns in the dataset
len(input_num_cols)

20

Pausing to do an initial sanity check here, we noticed that our initial dataset had 20 numerical columns (we checked the length of our `input_num_cols` array). 

Our from two cells above, we notice that our one hot encoding of the categorical columns resulted in 24 columns.

If we add 24 + 20, we obtain 44, which is consistent with the number of columns displayed when we printed `df_input_scaled_encoded`. 

We note here that the one hot encoded categorical columns are all appended at the end, and this is apparent when we displayed `df_input_scaled_encoded`. 

What we need to do next is figure out which column index corresponds to which category. E.g Column 30 refers to which category?


In [261]:
for i in range(len(input_num_cols)):
    df_input_scaled_encoded.rename(columns={df_input_scaled_encoded.columns[i]: input_num_cols[i]}, inplace = True)

for j in range(len(array_of_cat_titles)):
    df_input_scaled_encoded.rename(columns={df_input_scaled_encoded.columns[i+1]: array_of_cat_titles[j]}, inplace = True)
    i = i + 1


In [262]:
df_input_scaled_encoded.columns[43]

'x5_phosphinic acid'

In [263]:
df_input_scaled_encoded

,Temperature,Growth,Metal_amount (g),Metal_mmol (mmol),Metal_concentration (mmol/g),Chalcogen_amount (g),Chalcogen_mmol (mmol),Chalcogen_concentration (mmol/g),Metal/Se_ratio,CA_amount (g),...,x3_tributylphosphine,x3_trioctylphosphine,x3_triphenylphosphine,x4_None,x4_liquid parafin,x4_octadecene,x4_phenyl ether,x4_trioctylphosphine oxide,x5_None,x5_phosphinic acid
0,-0.539683,-0.08,-1.0000,-1.243697,-0.666667,0.000000,-0.111111,0.526316,-0.362745,-0.045437,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,-0.158730,0.40,-0.5000,-0.268908,4.333333,-0.365297,-0.419753,0.789474,1.024510,0.757289,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.158730,0.40,-0.5000,-0.268908,4.333333,-0.365297,-0.419753,0.789474,1.024510,0.757289,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.158730,0.40,-0.5000,-0.268908,4.333333,-0.365297,-0.419753,0.789474,1.024510,0.757289,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.158730,0.40,-0.5000,-0.268908,4.333333,-0.365297,-0.419753,0.789474,1.024510,0.757289,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,-0.349206,-0.16,1.1825,-0.672269,1.733333,-0.158904,2.753086,21.042105,-0.446078,-0.454373,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
108,-0.349206,-0.16,1.1825,-0.672269,1.733333,-0.158904,2.753086,21.042105,-0.446078,-0.454373,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
109,-0.349206,-0.16,1.1825,-0.672269,1.733333,-0.158904,2.753086,21.042105,-0.446078,-0.454373,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
110,-0.349206,-0.16,1.1825,-0.672269,1.733333,-0.158904,2.753086,21.042105,-0.446078,-0.454373,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [264]:
df_input_scaled_encoded.to_csv('CdSe Pandas Dataframe for ML Modelling (Scaled and Encoded)')